In [15]:
import json
import datetime
from pymongo import MongoClient
import pandas as pd

In [16]:
db_name = 'fleximart'
collection = 'products_catalog'
mongo_url = 'mongodb://localhost:27017'

In [17]:
import os
os.chdir(r'C:\Users\rites\OneDrive\Desktop\BITSOM\MODULE2\BITSoM_BA_25071329-fleximart-data-architecture')

In [18]:
import json
with open(file='products_catalog.json', mode='r', encoding='utf-8') as f:
    data = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'products_catalog.json'

In [ ]:
client = MongoClient(mongo_url)
db = client[db_name]
col = db[collection]

NameError: name 'mongo_url' is not defined

Operation 1: Load Data

In [19]:

rec = col.insert_many(data)
print(f"Inserted {len(rec.inserted_ids)} documents into {db_name}.{collection}")

NameError: name 'col' is not defined

Operation 2: Basic Query
Find all products in "Electronics" category with price less than 50000
Return only: name, price, stock

In [ ]:
q2 = col.find(
    {"category": "Electronics", "price": {"$lt": 50000}},
    {"_id": 0, "name": 1, "price": 1, "stock": 1},
)

pd.DataFrame(list(q2))

,name,price,stock
0,Sony WH-1000XM5 Headphones,29990.0,200
1,Dell 27-inch 4K Monitor,32999.0,60
2,OnePlus Nord CE 3,26999.0,180


Operation 3: Review Analysis (2 marks)
Find all products that have average rating >= 4.0
Use aggregation to calculate average from reviews array

In [ ]:
pipeline3 = [
    {
        "$addFields": {
            "avg_rating": {
                "$avg": {"$ifNull": ["$reviews.rating", []]}  # safe if no reviews
            }
        }
    },
    {"$match": {"avg_rating": {"$gte": 4.0}}},
    {
        "$project": {
            "_id": 0,
            "product_id": 1,
            "name": 1,
            "category": 1,
            "avg_rating": {"$round": ["$avg_rating", 2]},
        }
    },
]


pd.DataFrame(list(col.aggregate(pipeline3)))

,product_id,name,category,avg_rating
0,ELEC001,Samsung Galaxy S21 Ultra,Electronics,4.67
1,ELEC002,Apple MacBook Pro 14-inch,Electronics,5.00
2,ELEC003,Sony WH-1000XM5 Headphones,Electronics,4.67
3,ELEC004,Dell 27-inch 4K Monitor,Electronics,4.00
4,ELEC005,OnePlus Nord CE 3,Electronics,4.00
5,ELEC006,Samsung 55-inch QLED TV,Electronics,4.50
6,FASH001,Levi's 511 Slim Fit Jeans,Fashion,4.67
7,FASH002,Nike Air Max 270 Sneakers,Fashion,4.50
8,FASH003,Adidas Originals T-Shirt,Fashion,4.33
9,FASH004,Puma RS-X Sneakers,Fashion,4.50


Operation 4: Update Operation (2 marks)
Add a new review to product "ELEC001"
Review: {user: "U999", rating: 4, comment: "Good value", date: ISODate()}

In [ ]:
new_review = {
    "user": "U999",
    "rating": 4,
    "comment": "Good value",
    "date": datetime.datetime.now(),  # stored as BSON Date (like ISODate)
}

In [ ]:
res4 = col.update_one(
    {"product_id": "ELEC001"},
    {"$push": {"reviews": new_review}},
)
print({"matched": res4.matched_count, "modified": res4.modified_count})

{'matched': 1, 'modified': 1}


In [ ]:
updated = col.find_one(
    {"product_id": "ELEC001"},
    {"_id": 0, "product_id": 1, "name": 1, "reviews": 1},
)

updated

{'product_id': 'ELEC001',
 'name': 'Samsung Galaxy S21 Ultra',
 'reviews': [{'user_id': 'U001',
   'username': 'TechGuru',
   'rating': 5,
   'comment': 'Excellent phone with amazing camera quality!',
   'date': '2024-01-15'},
  {'user_id': 'U012',
   'username': 'MobileUser',
   'rating': 4,
   'comment': 'Great performance but a bit pricey.',
   'date': '2024-02-10'},
  {'user_id': 'U023',
   'username': 'PhotoEnthusiast',
   'rating': 5,
   'comment': "Best camera phone I've ever used!",
   'date': '2024-03-05'},
  {'user': 'U999',
   'rating': 4,
   'comment': 'Good value',
   'date': datetime.datetime(2026, 1, 4, 19, 42, 30, 71000)}]}

Operation 5: Complex Aggregation
Calculate average price by category
Return: category, avg_price, product_count
Sort by avg_price descending

In [ ]:
pipeline5 = [
    {
        "$group": {
            "_id": "$category",
            "avg_price": {"$avg": "$price"},
            "product_count": {"$sum": 1},
        }
    },
    {
        "$project": {
            "_id": 0,
            "category": "$_id",
            "avg_price": {"$round": ["$avg_price", 2]},
            "product_count": 1,
        }
    },
    {"$sort": {"avg_price": -1}},
]

pd.DataFrame(list(col.aggregate(pipeline5)))

,product_count,category,avg_price
0,6,Electronics,70830.83
1,6,Fashion,5215.00


In [ ]:
pd.read_json('products_catalog.json')

,product_id,name,category,subcategory,price,stock,specifications,reviews,tags,warranty_months,created_at,updated_at
0,ELEC001,Samsung Galaxy S21 Ultra,Electronics,Smartphones,79999,150,"{'brand': 'Samsung', 'ram': '12GB', 'storage':...","[{'user_id': 'U001', 'username': 'TechGuru', '...","[flagship, 5G, android, photography]",12,2023-12-01 10:00:00+00:00,2024-03-20 14:30:00+00:00
1,ELEC002,Apple MacBook Pro 14-inch,Electronics,Laptops,189999,45,"{'brand': 'Apple', 'processor': 'M2 Pro', 'ram...","[{'user_id': 'U005', 'username': 'DevPro', 'ra...","[laptop, macOS, professional, M2]",12,2023-11-15 09:00:00+00:00,2024-03-18 11:20:00+00:00
2,ELEC003,Sony WH-1000XM5 Headphones,Electronics,Audio,29990,200,"{'brand': 'Sony', 'type': 'Over-ear', 'connect...","[{'user_id': 'U007', 'username': 'MusicLover',...","[headphones, wireless, noise-cancelling, premium]",24,2023-10-20 08:00:00+00:00,2024-03-22 16:45:00+00:00
3,ELEC004,Dell 27-inch 4K Monitor,Electronics,Monitors,32999,60,"{'brand': 'Dell', 'screen_size': '27 inches', ...","[{'user_id': 'U003', 'username': 'GraphicDesig...","[monitor, 4K, professional, IPS]",36,2023-11-10 10:30:00+00:00,2024-03-15 09:15:00+00:00
4,ELEC005,OnePlus Nord CE 3,Electronics,Smartphones,26999,180,"{'brand': 'OnePlus', 'ram': '8GB', 'storage': ...","[{'user_id': 'U010', 'username': 'BudgetBuyer'...","[smartphone, mid-range, 5G, android, fast-char...",12,2024-01-05 11:00:00+00:00,2024-03-25 13:30:00+00:00
5,ELEC006,Samsung 55-inch QLED TV,Electronics,Televisions,64999,35,"{'brand': 'Samsung', 'screen_size': '55 inches...","[{'user_id': 'U008', 'username': 'MovieBuff', ...","[TV, QLED, 4K, smart-tv, 120Hz]",24,2023-12-15 09:30:00+00:00,2024-03-20 10:00:00+00:00
6,FASH001,Levi's 511 Slim Fit Jeans,Fashion,Clothing,3499,120,"{'brand': 'Levi's', 'material': '98% Cotton, 2...","[{'user_id': 'U002', 'username': 'FashionGuy',...","[jeans, denim, casual, mens-fashion]",3,2023-10-01 08:00:00+00:00,2024-03-18 14:20:00+00:00
7,FASH002,Nike Air Max 270 Sneakers,Fashion,Footwear,12995,85,"{'brand': 'Nike', 'type': 'Running Shoes', 'ma...","[{'user_id': 'U004', 'username': 'RunnerLife',...","[shoes, sneakers, running, athletic, nike]",6,2023-11-20 10:00:00+00:00,2024-03-22 11:30:00+00:00
8,FASH003,Adidas Originals T-Shirt,Fashion,Clothing,1499,200,"{'brand': 'Adidas', 'material': '100% Cotton',...","[{'user_id': 'U006', 'username': 'CasualStyle'...","[t-shirt, casual, cotton, sportswear]",3,2023-12-10 09:00:00+00:00,2024-03-23 15:10:00+00:00
9,FASH004,Puma RS-X Sneakers,Fashion,Footwear,8999,95,"{'brand': 'Puma', 'type': 'Casual Sneakers', '...","[{'user_id': 'U009', 'username': 'StreetStyle'...","[sneakers, casual, retro, streetwear]",6,2024-01-08 11:30:00+00:00,2024-03-24 12:45:00+00:00
